In [1]:
import numpy as np
import ioh
import math
import random

In [2]:
ioh.ProblemClass.GRAPH.problems

{2000: 'MaxCut2000',
 2001: 'MaxCut2001',
 2002: 'MaxCut2002',
 2003: 'MaxCut2003',
 2004: 'MaxCut2004',
 2100: 'MaxCoverage2100',
 2101: 'MaxCoverage2101',
 2102: 'MaxCoverage2102',
 2103: 'MaxCoverage2103',
 2104: 'MaxCoverage2104',
 2105: 'MaxCoverage2105',
 2106: 'MaxCoverage2106',
 2107: 'MaxCoverage2107',
 2108: 'MaxCoverage2108',
 2109: 'MaxCoverage2109',
 2110: 'MaxCoverage2110',
 2111: 'MaxCoverage2111',
 2112: 'MaxCoverage2112',
 2113: 'MaxCoverage2113',
 2114: 'MaxCoverage2114',
 2115: 'MaxCoverage2115',
 2116: 'MaxCoverage2116',
 2117: 'MaxCoverage2117',
 2118: 'MaxCoverage2118',
 2119: 'MaxCoverage2119',
 2120: 'MaxCoverage2120',
 2121: 'MaxCoverage2121',
 2122: 'MaxCoverage2122',
 2123: 'MaxCoverage2123',
 2124: 'MaxCoverage2124',
 2125: 'MaxCoverage2125',
 2126: 'MaxCoverage2126',
 2127: 'MaxCoverage2127',
 2200: 'MaxInfluence2200',
 2201: 'MaxInfluence2201',
 2202: 'MaxInfluence2202',
 2203: 'MaxInfluence2203',
 2204: 'MaxInfluence2204',
 2205: 'MaxInfluence2205',
 2206

In [3]:
max_cut = ioh.get_problem(2000, problem_class=ioh.ProblemClass.GRAPH)
max_coverage = ioh.get_problem(2100, problem_class=ioh.ProblemClass.GRAPH)

In [4]:
print(max_cut.meta_data)
print(max_coverage.meta_data)

<MetaData: MaxCut2000 id: 2000 iid: 1 dim: 800>
<MetaData: MaxCoverage2100 id: 2100 iid: 1 dim: 450>


In [5]:
nb_it = 10000

# Multi-objective optimization

In [6]:
import random
from typing import Callable

In [7]:
class POSS:
    
    def __init__(self, n: int, pflip: int, b: int, I: Callable):
        self.n: int = n
        self.pflip = pflip #proba flit un bit
        self.b: int = b #budget
        self.I: Callable = I #isolation function : two solutions are comparable if they have the same I value
        
    def __call__(self, problem: ioh.ProblemClass.GRAPH) -> None:

        def constraint1(s):
            nb1 = np.count_nonzero(s[0])
            if nb1 >= 2*self.b: return -100000 # -INFINI
            return s[1]
        
        def constraint2(s):
            return -np.count_nonzero(s[0])
        
        def leq(x,y): # <=
            if constraint1(x) <= constraint1(y):
                if constraint2(x) <= constraint2(y):
                    return True
                    
        def lt(x,y):  # <
            if constraint1(x) < constraint1(y):
                if constraint2(x) <= constraint2(y):
                    return True
            if constraint1(x) <= constraint1(y):
                if constraint2(x) < constraint2(y):
                    return True
        
        def bitwise_mutation(s):
            x = s[0].copy()

            for i in range(problem.meta_data.n_variables):
                flip = 1 if random.random() < self.pflip else 0
                if flip:
                    if x[i]: x[i] = 0
                    else: x[i] = 1
            
            return (x,problem(x))
        
        s0 = np.zeros(problem.meta_data.n_variables).astype(int)
        s = (s0,problem(s0))
        P = [(s[0].copy(), s[1])]
        for t in range(self.n):
            if not t%100:
                print("Iteration : ", t)
            s = P[np.random.randint(len(P))] #Select a solution from P uar            
            ns = bitwise_mutation(s)
            
            better_than_ns = [z for z in P if self.I(z[0])==self.I(ns[0]) and lt(ns,z)]

            if not better_than_ns :
                Q = [z for z in P if self.I(z)==self.I(ns) and leq(z,ns)]
                P = [z for z in P if not any(np.array_equal(z, q) for q in Q)]
                P.append((ns[0].copy(),ns[1]))

In [15]:
import math
max_coverage.reset()
dim_pb = max_coverage.meta_data.n_variables
poss = POSS(1000, 1/dim_pb, 10, lambda x: np.count_nonzero(x[0]))
poss(max_coverage)

Iteration :  0
Iteration :  100


/home/buzetq/.local/lib/python3.8/site-packages/numpy/core/numeric.py:2457: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  a1, a2 = asarray(a1), asarray(a2)


Iteration :  200
Iteration :  300
Iteration :  400
Iteration :  500
Iteration :  600
Iteration :  700
Iteration :  800
Iteration :  900


In [16]:
max_coverage.state

<State evaluations: 1001 optimum_found: false current_best: <Solution x: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

# Algorithme par croissance

In [8]:
def flip(x,l):
    positions = np.random.choice(np.arange(0,len(x)), l, replace=False)
    y = x.copy()
    y[positions] = 1 - y[positions]
    return y

In [9]:
class Test:
    
    def __init__(self, n: int, s: int, k:int):
        self.n: int = n #nb d'itérations
        self.s: int = s #population size
        self.k: int = k #taux de croissance
    
    def __call__(self, problem: ioh.ProblemClass.GRAPH) -> None:
        dim = problem.meta_data.n_variables
        
        #Initialization
        x = np.zeros(dim).astype(int)
        y = problem(x)
        
        #Optimization
        for t in range(self.n-1):
            #On tire k échantillons et on garde celui qui fait le plus croitre la fonction
            pop = []
            for i in range(self.s):
                xx = flip(x,self.k)
                yy = problem(xx)
                pop.append((xx,yy))
            pop.sort(key=lambda x: x[1])
            if pop[self.s-1][1] > y:
                x = pop[self.s-1][0]
                y = pop[self.s-1][1]
            else:                    
                self.k = max(int(self.k*3/4),1)

In [10]:
max_cut.reset()
dim = max_cut.meta_data.n_variables
test = Test(nb_it//10, 10, 25)
#test = Test(nb_it//1000, 1000, 5)
test(max_cut)
max_cut.state

<State evaluations: 9991 optimum_found: false current_best: <Solution x: [1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0,

In [18]:
max_coverage.reset()
dim = max_coverage.meta_data.n_variables
test = Test(nb_it//1000, 1000, 5)
#test = Test(nb_it//10, 10, 25)
test(max_coverage)
max_coverage.state

<State evaluations: 9001 optimum_found: false current_best: <Solution x: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [103]:
class TestReset:
    
    def __init__(self, n: int, s: int, k:int):
        self.n: int = n #nb d'itérations
        self.s: int = s #population size
        self.k: int = k #taux de croissance
    
    def __call__(self, problem: ioh.ProblemClass.GRAPH) -> None:
        dim = problem.meta_data.n_variables
        
        #Initialization
        x = np.zeros(dim).astype(int)
        y0 = problem(x)
        y = y0
        self.k0 = self.k
        comp = 0

        #Optimization
        for t in range(self.n-1):            
            #On tire k échantillons et on garde celui qui fait le plus croitre la fonction
            pop = []
            for i in range(self.s):
                xx = flip(x,self.k)
                yy = problem(xx)
                pop.append((xx,yy))
            pop.sort(key=lambda x: x[1])
            if pop[self.s-1][1] > y:
                x = pop[self.s-1][0]
                y = pop[self.s-1][1]
            else:        
                if self.k == 1:
                    comp += 1

                self.k = max(int(self.k*3/4),1)
                
                if self.k == 1 and comp == 10: #reset
                    self.k = self.k0
                    x = np.zeros(dim).astype(int)
                    y = y0
                    comp = 0

In [108]:
max_cut.reset()
dim = max_cut.meta_data.n_variables
test = TestReset(nb_it//10, 10, 25)
test(max_cut)
max_cut.state

<State evaluations: 9991 optimum_found: false current_best: <Solution x: [0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0,

In [109]:
max_coverage.reset()
dim = max_coverage.meta_data.n_variables
test = TestReset(nb_it//1000, 1000, 5)
test(max_coverage)
max_coverage.state

<State evaluations: 9001 optimum_found: false current_best: <Solution x: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

# Algorithme par décroissance

In [33]:
class Test3:
    
    def __init__(self, n: int, s: int, k:int):
        self.n: int = n #nb d'itérations
        self.s: int = s #population size
        self.k: int = k #taux de croissance
    
    def __call__(self, problem: ioh.ProblemClass.GRAPH) -> None:
        dim = problem.meta_data.n_variables
        
        #Initialization
        x = np.ones(dim).astype(int)
        y = problem(x)
        
        #Optimization
        for t in range(self.n-1):
            #On tire k échantillons et on garde celui qui fait le plus croitre la fonction
            pop = []
            for i in range(self.s):
                xx = flip(x,self.k)
                yy = problem(xx)
                pop.append((xx,yy))
            pop.sort(key=lambda x: x[1])
            if random.random() < self.s/dim or pop[self.s-1][1] > y:
                x = pop[self.s-1][0]
                y = pop[self.s-1][1]
                self.k = min(int(self.k*4/3),dim)
            else:
                self.k = max(int(self.k*3/4),1)

In [34]:
max_cut.reset()
dim = max_cut.meta_data.n_variables
test3 = Test3(nb_it//10, 10, 25)
test3(max_cut)
max_cut.state

<State evaluations: 9991 optimum_found: false current_best: <Solution x: [1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0,

In [283]:
max_coverage.reset()
dim = max_coverage.meta_data.n_variables
test3 = Test3(nb_it//1000, 1000, 5)
test3(max_coverage)
max_coverage.state

<State evaluations: 9001 optimum_found: false current_best: <Solution x: [0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

# Combinaison d'algorithmes

In [39]:
def positive_binomial(n,p,nb_samples):
    gen = []
    for i in range(nb_samples):
        trial = np.random.binomial(n, p, 1)[0]
        while trial == 0:
            trial = np.random.binomial(n, p, 1)[0]
        gen.append(trial)
    return gen

In [256]:
def UMDA(problem,n,s):
    dim = problem.meta_data.n_variables
    
    def sample(p):
        x = np.zeros(dim,int)
        for j in range(dim):
            x[j] = 1 if random.random() < p[j] else 0
        y = problem(x)
        return (x,y)
    
    #Initialization
    z = []
    for i in range(s):
        x = np.random.randint(0,2,dim)
        y = problem(x)
        z.append((x,y))
    z.sort(key=lambda x: x[1])
    P = np.array(z[s-s//2:])
    
    #Optimization
    p = np.zeros(dim)
    for t in range(1,n):
        for j in range(dim):
            count = 0
            for k in range(s//2):
                if P[k,0][j] == 1: count +=1
            
            p[j] = 2*count/s
            if p[j] < 1/dim: p[j] = 1/dim
            if p[j] > 1 - 1/dim: p[j] = 1 - 1/dim
    
        for i in range(s):
            z[i] = sample(p)
        z.sort(key=lambda x: x[1])
        P = np.array(z[s-s//2:])
        
    return P

In [259]:
class ULog: #UMDA + EAlog

    def __init__(self,n,lamb):
        self.n = n
        self.lamb = lamb

    def __call__(self, problem: ioh.ProblemClass.GRAPH) -> None:
        dim = problem.meta_data.n_variables
        
        #Initialization
        U = UMDA(problem,10,5)
        x = U[len(U)-1][0]
        y = U[len(U)-1][1]
        p = 0.2
        
        #Optimization
        for t in range(self.n-1):
            pop = []
            for i in range(self.lamb):
                sample = np.random.normal(0,1)
                pp = 1/(1 + (1-p)/p * math.exp(0.22 * sample))
                l = positive_binomial(dim,pp,1)[0]
                xx = flip(x,l)
                yy = problem(xx)
                pop.append((xx,yy,pp))
                
            pop.sort(key=lambda x: x[1])
            p = pop[self.lamb-1][2]
            if pop[self.lamb-1][1] >= y:
                x = pop[self.lamb-1][0]
                y = pop[self.lamb-1][1]

In [263]:
max_coverage.reset()
dim = max_coverage.meta_data.n_variables
ulog = ULog(nb_it//10, 10)
ulog(max_coverage)
max_coverage.state

/tmp/ipykernel_15343/782673433.py:18: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  P = np.array(z[s-s//2:])
/tmp/ipykernel_15343/782673433.py:35: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  P = np.array(z[s-s//2:])


<State evaluations: 10040 optimum_found: false current_best: <Solution x: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0

In [266]:
max_cut.reset()
dim = max_cut.meta_data.n_variables
ulog = ULog(nb_it//10, 10)
ulog(max_cut)
max_cut.state

/tmp/ipykernel_15343/782673433.py:18: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  P = np.array(z[s-s//2:])
/tmp/ipykernel_15343/782673433.py:35: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  P = np.array(z[s-s//2:])


<State evaluations: 10040 optimum_found: false current_best: <Solution x: [1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1

# Crossover par Moyenne

In [209]:
def gen_budget(b,N):
    pop = []
    for i in range(N):
        x = np.zeros(dim).astype(int)
        iflips = np.random.randint(0,dim,b)
        x[iflips] = 1
        y = problem(x)
        z = (x,y)
        pop.append(z)
    return pop

In [243]:
class Test2:
    def __init__(self, n: int, s: int, p: float):
        self.n: int = n #Number of iterations
        self.s: int = s #Population size
        self.p: float = p #Proba bitflip
    
    def __call__(self, problem: ioh.ProblemClass.GRAPH) -> None:
        dim = problem.meta_data.n_variables
        
        def mutation(x):
            y = x.copy()
            for i in range(dim):
                y[i] = 1 - y[i] if random.random() < self.p else y[i]
            return y
            
        def crossover(pop):
            #Calcul de la distribution de probabilités 
            dist = np.zeros(dim)
            normalization = 0
            for i in range(self.s):
                dist = dist + pop[i][0] * pop[i][1]
                normalization += pop[i][1]
            dist = dist / normalization
            #Génération de la nouvelle population
            npop = []
            for t in range(self.s):
                x = np.zeros(dim).astype(int)
                for i in range(dim):
                    x[i] = 1 if random.random() < dist[i] else 0
                y = problem(x)
                z = (x,y)
                npop.append(z)
            return npop
        
        #Initialization
        x1 = np.zeros(dim).astype(int)
        y1 = problem(x1)
        x2 = np.random.randint(0,2,dim)
        y2 = problem(x2)
        x = x1 if y1 >= y2 else x2
        y = y1 if y1 >= y2 else y2

        #Optimization
        for t in range(self.n-2):
            pop = []
            for i in range(self.s):
                xx = mutation(x)
                yy = problem(xx)
                pop.append((xx,yy))
                
            pop = crossover(pop)
            
            pop.sort(key=lambda x: x[1])
            if pop[self.s-1][1] >= y:
                x = pop[self.s-1][0]
                y = pop[self.s-1][1]


In [244]:
max_cut.reset()
dim = max_cut.meta_data.n_variables
test2 = Test2(nb_it//(2*10), 10, 1/dim)
test2(max_cut)

In [245]:
max_cut.state

<State evaluations: 9962 optimum_found: false current_best: <Solution x: [1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1,

In [228]:
max_coverage.reset()
dim = max_coverage.meta_data.n_variables
test2 = Test2(nb_it//(2*10), 10, 1/dim)
test2(max_coverage)

In [229]:
max_coverage.state

<State evaluations: 9962 optimum_found: false current_best: <Solution x: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,

In [ ]:
import math
max_cut.reset()
dim = max_cut.meta_data.n_variables
test2 = Test2(1000, 10, 1/dim, 400)
test2(max_cut)

In [ ]:
max_cut.state

# Big jump

In [289]:
np.random.randint(0, 10,2)

array([0, 1])

In [291]:
-1%4

3

In [40]:
def select_distant_indices(array_length, nb, min_dist, proba_pursue):
    selected_indices = [np.random.randint(0, array_length)]

    while len(selected_indices) < nb :
        new_index = np.random.randint(0, array_length)
        ok = True
        for index in selected_indices:
            d1 = abs(new_index - index)
            d2 = (-abs(new_index - index))%array_length
            if d1 <= min_dist or d2 <= min_dist:
                if random.random() > proba_pursue:
                    ok = False
        if ok: selected_indices.append(new_index)

    return selected_indices

In [41]:
def flip_set(x,positions):
    y = x.copy()
    y[positions] = 1 - y[positions]
    return y

In [42]:
class EABJ:

    def __init__(self,n,lamb,min_dist,proba_pursue):
        self.n = n #nb iterations
        self.lamb = lamb #population size
        self.min_dist = min_dist
        self.proba_pursue = proba_pursue
    
    def __call__(self, problem: ioh.ProblemClass.GRAPH) -> None:
        dim = problem.meta_data.n_variables

        #Initialization
        x = np.random.randint(0,2,dim)
        y = problem(x)

        #Optimization
        for t in range(self.n-1):
            pop = []
            for i in range(self.lamb):
                l = positive_binomial(dim,3/dim,1)[0]
                I = select_distant_indices(dim, l, self.min_dist, self.proba_pursue)
                xx = flip_set(x,I)
                yy = problem(xx)
                pop.append((xx,yy))
            pop.sort(key=lambda x: x[1])
            if pop[self.lamb-1][1] >= y:
                x = pop[self.lamb-1][0]
                y = pop[self.lamb-1][1]

In [43]:
max_cut.reset()
dim = max_cut.meta_data.n_variables
eabj = EABJ(nb_it//10, 10, 4, 1/dim)
eabj(max_cut)
max_cut.state

<State evaluations: 9991 optimum_found: false current_best: <Solution x: [0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0,

In [44]:
max_coverage.reset()
dim = max_coverage.meta_data.n_variables
eabj = EABJ(nb_it//10, 10, 5, 10/dim)
eabj(max_coverage)
max_coverage.state

<State evaluations: 9991 optimum_found: false current_best: <Solution x: [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

# Méta-optimisation

In [12]:
def point_crossover(x1,x2,dim):
    j = np.random.randint(0,dim)
    parent1 = x1.copy()
    parent2 = x2.copy()
    offspring1 = np.concatenate((parent1[:j],parent2[j:]))
    offspring2 = np.concatenate((parent2[:j],parent1[j:]))
    return offspring1, offspring2

In [52]:
class VanillaGA:

    def __init__(self,n,mu,pm,pc):
        self.n = n #Nb iterations
        self.mu = mu #Population size
        self.pm = pm #Muration rate
        self.pc = pc #Crossover probability

    @classmethod
    def from_list(cls, args_list):
        return cls(*args_list)
    
    def __call__(self,problem: ioh.ProblemClass.GRAPH) -> None:
        dim = problem.meta_data.n_variables
        
        #Initialization
        pop = []
        for i in range(self.mu):
            x = np.random.randint(0,2,dim)
            y = problem(x)
            pop.append((x,y))
        
        #Optimization
        for t in range(self.n-1):
            #Parent selection phase (roulette-wheel selection)
            normalization = 0
            for i in range(self.mu):
                normalization += pop[i][1]
            probas = [pop[i][1]/normalization for i in range(self.mu)]
            index = np.random.choice(self.mu,self.mu,replace=True,p=probas)
            pop = np.array(pop)
            npop = pop[index]
            
            #Crossover phase
            for i in range(self.mu//2):
                if random.random() <= self.pc:
                    offspring1, offspring2 = point_crossover(npop[i][0],npop[2*i][0],dim)
                    npop[i][0] = offspring1
                    npop[2*i][0] = offspring2

            #Mutation phase
            for i in range(self.mu):
                l = np.random.binomial(dim,self.pm) #Positive binomial better?
                npop[i][0] = flip(npop[i][0],l)
                npop[i][1] = problem(npop[i][0])
            
            #Replacement
            for i in range(self.mu):
                pop[i] = npop[i]

In [53]:
max_cut.reset()
dim = max_cut.meta_data.n_variables
vga = VanillaGA.from_list([nb_it//10,10,1/dim,1/10])
vga(max_cut)
print(max_cut.state)

/tmp/ipykernel_4385/469637379.py:31: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  pop = np.array(pop)


<State evaluations: 10000 optimum_found: false current_best: <Solution x: [0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1

In [78]:
from enum import Enum

class ParamConstraint(Enum):
    FIX = 0
    POPULATION = 1
    INTEGER = 2
    REAL = 3

# (1+1) EA pour optimiser un algorithme génétique
class MetaOptimizer:

    def __init__(self, algo, n, lamb, set, ref, init, constraint_types, bounds):
        self.algo = algo #algorithme à optimiser
        self.lamb = lamb #population
        
        self.n = n #nb d'itérations
        
        self.set = set #ens des problèmes à optimiser
        self.ref = ref #valeur de réf pour les problèmes à optimiser
        
        self.init = init #paramètres de départ
        self.constraint_types = constraint_types #type chaque contrainte  [INTEGER,FLOAT]
        self.bounds = bounds #encadrements sur les paramètres de l'algo [[1,2], [2.05,42.62]]

    def load_problems(self):
        problems = []
        for i in range(len(self.set)): 
            problems.append(ioh.get_problem(self.set[i], problem_class=ioh.ProblemClass.GRAPH))
            problems[i].reset()
        return problems

    def eval_params(self,parameters):
        scores = np.zeros(len(self.set))
        problems = self.load_problems()
        for i in range(len(self.set)):
            instance = self.algo.from_list(parameters)
            instance(problems[i])
            scores[i] = problems[i].state.y_unconstrained_best
        scores = (scores - ref) / ref
        score = scores.sum()
        return score

    def __call__(self) -> None:

        #Génère un nouveau point à partir du précédent en modifiant certaines coordonnées par un tirage aléatoire
        def sample_from(x,l,constraint_types,bounds):
            y = x.copy()
            
            ind = np.random.randint(0,len(constraint_types),l) #Paramètres à modifier (pas le nb d'itérations)
            ok = False
            while not ok:
                ok = True
                for i in ind:
                    if constraint_types[i] == ParamConstraint.FIX:
                        ok = False
                        ind = np.random.randint(0,len(constraint_types),l)
                        
            for i in ind:
                if constraint_types[i] == ParamConstraint.POPULATION: #Le nb d'itérations dépend de la taille de la population
                    y[i] = np.random.randint(bounds[i][0],bounds[i][1]+1)
                    y[0] = nb_it // y[i]
                if constraint_types[i] == ParamConstraint.INTEGER:
                    y[i] = np.random.randint(bounds[i][0],bounds[i][1]+1)
                if constraint_types[i] == ParamConstraint.REAL:
                    y[i] = np.random.uniform(bounds[i][0],bounds[i][1])
            return y            
        
        dim = len(self.set)
        
        #Initialization
        self.params = init
        self.best_score = self.eval_params(init)

        #Optimization
        for t in range(self.n-1):
            print(t)
            pop = []
            for i in range(self.lamb):
                l = positive_binomial(dim,1/dim,1)
                xx = sample_from(self.params,l,self.constraint_types,self.bounds)
                yy = self.eval_params(xx)
                pop.append((xx,yy))
            pop.sort(key=lambda x: x[1])
            if pop[self.lamb-1][1] >= self.best_score:
                self.params = pop[self.lamb-1][0]
                self.best_score = pop[self.lamb-1][1]


In [79]:
max_cut.reset()

In [80]:
algo = VanillaGA
n = 1000
lamb = 1
set = [2000,2001,2002,2003,2004]
ref = [10000,10000,10000,10000,10000]
init = [nb_it//10,10,1/dim,1/10]
constraint_types = [ParamConstraint.FIX, ParamConstraint.POPULATION, ParamConstraint.REAL, ParamConstraint.REAL]
bounds = [[nb_it,nb_it],[1,nb_it//10],[0,1],[0,1]]
mo = MetaOptimizer(algo, n, lamb, set, ref, init, constraint_types, bounds)
mo()

/tmp/ipykernel_4385/469637379.py:31: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  pop = np.array(pop)


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [81]:
mo.best_score

-0.0577

In [82]:
mo.params

[13, 760, 0.5647327183344031, 0.3357726225619695]

In [83]:
max_cut.reset()
dim = max_cut.meta_data.n_variables
vga = VanillaGA.from_list(mo.params)
vga(max_cut)
print(max_cut.state)

/tmp/ipykernel_4385/469637379.py:31: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  pop = np.array(pop)


<State evaluations: 9880 optimum_found: false current_best: <Solution x: [1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1,

In [85]:
max_cut1 = ioh.get_problem(2001, problem_class=ioh.ProblemClass.GRAPH)
max_cut2 = ioh.get_problem(2002, problem_class=ioh.ProblemClass.GRAPH)
max_cut3 = ioh.get_problem(2003, problem_class=ioh.ProblemClass.GRAPH)
max_cut4 = ioh.get_problem(2004, problem_class=ioh.ProblemClass.GRAPH)

In [86]:
max_cut1.reset()
max_cut2.reset()
max_cut3.reset()
max_cut4.reset()
vga(max_cut1)
print(max_cut1.state)
vga(max_cut2)
print(max_cut2.state)
vga(max_cut2)
print(max_cut2.state)
vga(max_cut3)
print(max_cut3.state)
vga(max_cut4)
print(max_cut4.state)

/tmp/ipykernel_4385/469637379.py:31: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  pop = np.array(pop)


<State evaluations: 9880 optimum_found: false current_best: <Solution x: [1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1,

# Résultats

In [12]:
import os

In [13]:
def run_experiment(problem, algorithm, n_runs):
    for run in range(n_runs):
        algorithm(problem)
        print(problem.meta_data.name,problem.state.evaluations, problem.state.y_unconstrained_best)
        problem.reset()

#### Algorithme par croissance

In [25]:
logger_grow = ioh.logger.Analyzer(
    root=os.getcwd(),                  # Store data in the current working directory
    folder_name="grow10-25_maxcut",       # in a folder named: 'my-experiment'
    algorithm_name="grow10-25_ea",    # meta-data for the algorithm used to generate these results
    store_positions=True               # store x-variables in the logged files
)

for i in range(2000,2005):
    problem = ioh.get_problem(i, problem_class=ioh.ProblemClass.GRAPH)
    problem.attach_logger(logger_grow)
    dim_pb = problem.meta_data.n_variables
    grow = Test(nb_it//10, 10, 25)
    run_experiment(problem, grow, n_runs=10)

logger_grow.close()

MaxCut2000 9991 11319.0
MaxCut2000 9991 11331.0
MaxCut2000 9991 11281.0
MaxCut2000 9991 11338.0
MaxCut2000 9991 11326.0
MaxCut2000 9991 11287.0
MaxCut2000 9991 11289.0
MaxCut2000 9991 11330.0
MaxCut2000 9991 11323.0
MaxCut2000 9991 11304.0
MaxCut2001 9991 11300.0
MaxCut2001 9991 11323.0
MaxCut2001 9991 11305.0
MaxCut2001 9991 11343.0
MaxCut2001 9991 11316.0
MaxCut2001 9991 11295.0
MaxCut2001 9991 11270.0
MaxCut2001 9991 11286.0
MaxCut2001 9991 11286.0
MaxCut2001 9991 11290.0
MaxCut2002 9991 11362.0
MaxCut2002 9991 11369.0
MaxCut2002 9991 11353.0
MaxCut2002 9991 11285.0
MaxCut2002 9991 11295.0
MaxCut2002 9991 11341.0
MaxCut2002 9991 11301.0
MaxCut2002 9991 11292.0
MaxCut2002 9991 11337.0
MaxCut2002 9991 11355.0
MaxCut2003 9991 11341.0
MaxCut2003 9991 11363.0
MaxCut2003 9991 11321.0
MaxCut2003 9991 11302.0
MaxCut2003 9991 11299.0
MaxCut2003 9991 11247.0
MaxCut2003 9991 11324.0
MaxCut2003 9991 11339.0
MaxCut2003 9991 11314.0
MaxCut2003 9991 11290.0
MaxCut2004 9991 11382.0
MaxCut2004 9991 

In [26]:
logger_grow = ioh.logger.Analyzer(
    root=os.getcwd(),                  # Store data in the current working directory
    folder_name="grow10-10_maxcut",       # in a folder named: 'my-experiment'
    algorithm_name="grow10-10_ea",    # meta-data for the algorithm used to generate these results
    store_positions=True               # store x-variables in the logged files
)

for i in range(2000,2005):
    problem = ioh.get_problem(i, problem_class=ioh.ProblemClass.GRAPH)
    problem.attach_logger(logger_grow)
    dim_pb = problem.meta_data.n_variables
    grow = Test(nb_it//10, 10, 10)
    run_experiment(problem, grow, n_runs=10)

logger_grow.close()

MaxCut2000 9991 11366.0
MaxCut2000 9991 11319.0
MaxCut2000 9991 11345.0
MaxCut2000 9991 11281.0
MaxCut2000 9991 11299.0
MaxCut2000 9991 11300.0
MaxCut2000 9991 11411.0
MaxCut2000 9991 11279.0
MaxCut2000 9991 11294.0
MaxCut2000 9991 11240.0
MaxCut2001 9991 11300.0
MaxCut2001 9991 11326.0
MaxCut2001 9991 11271.0
MaxCut2001 9991 11280.0
MaxCut2001 9991 11359.0
MaxCut2001 9991 11312.0
MaxCut2001 9991 11352.0
MaxCut2001 9991 11336.0
MaxCut2001 9991 11386.0
MaxCut2001 9991 11304.0
MaxCut2002 9991 11294.0
MaxCut2002 9991 11351.0
MaxCut2002 9991 11333.0
MaxCut2002 9991 11319.0
MaxCut2002 9991 11289.0
MaxCut2002 9991 11319.0
MaxCut2002 9991 11290.0
MaxCut2002 9991 11358.0
MaxCut2002 9991 11368.0
MaxCut2002 9991 11358.0
MaxCut2003 9991 11333.0
MaxCut2003 9991 11296.0
MaxCut2003 9991 11381.0
MaxCut2003 9991 11319.0
MaxCut2003 9991 11329.0
MaxCut2003 9991 11354.0
MaxCut2003 9991 11359.0
MaxCut2003 9991 11260.0
MaxCut2003 9991 11329.0
MaxCut2003 9991 11357.0
MaxCut2004 9991 11292.0
MaxCut2004 9991 

In [27]:
logger_grow = ioh.logger.Analyzer(
    root=os.getcwd(),                  # Store data in the current working directory
    folder_name="grow100-10_maxcut",       # in a folder named: 'my-experiment'
    algorithm_name="grow100-10_ea",    # meta-data for the algorithm used to generate these results
    store_positions=True               # store x-variables in the logged files
)

for i in range(2000,2005):
    problem = ioh.get_problem(i, problem_class=ioh.ProblemClass.GRAPH)
    problem.attach_logger(logger_grow)
    dim_pb = problem.meta_data.n_variables
    grow = Test(nb_it//100, 100, 10)
    run_experiment(problem, grow, n_runs=10)

logger_grow.close()

MaxCut2000 9901 10779.0
MaxCut2000 9901 10152.0
MaxCut2000 9901 10110.0
MaxCut2000 9901 10128.0
MaxCut2000 9901 10161.0
MaxCut2000 9901 10152.0
MaxCut2000 9901 10212.0
MaxCut2000 9901 10219.0
MaxCut2000 9901 10127.0
MaxCut2000 9901 10146.0
MaxCut2001 9901 10844.0
MaxCut2001 9901 10150.0
MaxCut2001 9901 10074.0
MaxCut2001 9901 10133.0
MaxCut2001 9901 10114.0
MaxCut2001 9901 10112.0
MaxCut2001 9901 10158.0
MaxCut2001 9901 10100.0
MaxCut2001 9901 10110.0
MaxCut2001 9901 10094.0
MaxCut2002 9901 10738.0
MaxCut2002 9901 10203.0
MaxCut2002 9901 10154.0
MaxCut2002 9901 10154.0
MaxCut2002 9901 10161.0
MaxCut2002 9901 10153.0
MaxCut2002 9901 10064.0
MaxCut2002 9901 10136.0
MaxCut2002 9901 10172.0
MaxCut2002 9901 10097.0
MaxCut2003 9901 10834.0
MaxCut2003 9901 10155.0
MaxCut2003 9901 10202.0
MaxCut2003 9901 10134.0
MaxCut2003 9901 10124.0
MaxCut2003 9901 10118.0
MaxCut2003 9901 10180.0
MaxCut2003 9901 10106.0
MaxCut2003 9901 10121.0
MaxCut2003 9901 10155.0
MaxCut2004 9901 10835.0
MaxCut2004 9901 

In [28]:
logger_grow = ioh.logger.Analyzer(
    root=os.getcwd(),                  # Store data in the current working directory
    folder_name="grow1000-10_maxcut",       # in a folder named: 'my-experiment'
    algorithm_name="grow1000-10_ea",    # meta-data for the algorithm used to generate these results
    store_positions=True               # store x-variables in the logged files
)

for i in range(2000,2004+1):
    problem = ioh.get_problem(i, problem_class=ioh.ProblemClass.GRAPH)
    problem.attach_logger(logger_grow)
    dim_pb = problem.meta_data.n_variables
    grow = Test(nb_it//1000, 1000, 10)
    run_experiment(problem, grow, n_runs=10)

logger_grow.close()

MaxCut2000 9001 4271.0
MaxCut2000 9001 4314.0
MaxCut2000 9001 4292.0
MaxCut2000 9001 4310.0
MaxCut2000 9001 4352.0
MaxCut2000 9001 4307.0
MaxCut2000 9001 4305.0
MaxCut2000 9001 4296.0
MaxCut2000 9001 4264.0
MaxCut2000 9001 4320.0
MaxCut2001 9001 4277.0
MaxCut2001 9001 4308.0
MaxCut2001 9001 4300.0
MaxCut2001 9001 4327.0
MaxCut2001 9001 4381.0
MaxCut2001 9001 4303.0
MaxCut2001 9001 4259.0
MaxCut2001 9001 4330.0
MaxCut2001 9001 4313.0
MaxCut2001 9001 4286.0
MaxCut2002 9001 4283.0
MaxCut2002 9001 4316.0
MaxCut2002 9001 4299.0
MaxCut2002 9001 4312.0
MaxCut2002 9001 4341.0
MaxCut2002 9001 4361.0
MaxCut2002 9001 4332.0
MaxCut2002 9001 4318.0
MaxCut2002 9001 4359.0
MaxCut2002 9001 4292.0
MaxCut2003 9001 4299.0
MaxCut2003 9001 4329.0
MaxCut2003 9001 4300.0
MaxCut2003 9001 4343.0
MaxCut2003 9001 4299.0
MaxCut2003 9001 4322.0
MaxCut2003 9001 4316.0
MaxCut2003 9001 4312.0
MaxCut2003 9001 4303.0
MaxCut2003 9001 4316.0
MaxCut2004 9001 4307.0
MaxCut2004 9001 4294.0
MaxCut2004 9001 4351.0
MaxCut2004 

In [29]:
logger_grow = ioh.logger.Analyzer(
    root=os.getcwd(),                  # Store data in the current working directory
    folder_name="grow1000-5_maxcoverage",       # in a folder named: 'my-experiment'
    algorithm_name="grow1000-5_ea",    # meta-data for the algorithm used to generate these results
    store_positions=True               # store x-variables in the logged files
)

for i in range(2100,2127+1):
    problem = ioh.get_problem(i, problem_class=ioh.ProblemClass.GRAPH)
    problem.attach_logger(logger_grow)
    dim_pb = problem.meta_data.n_variables
    grow = Test(nb_it//1000, 1000, 10)
    run_experiment(problem, grow, n_runs=10)

logger_grow.close()

MaxCoverage2100 9001 429.0
MaxCoverage2100 9001 433.0
MaxCoverage2100 9001 433.0
MaxCoverage2100 9001 433.0
MaxCoverage2100 9001 435.0
MaxCoverage2100 9001 436.0
MaxCoverage2100 9001 432.0
MaxCoverage2100 9001 434.0
MaxCoverage2100 9001 435.0
MaxCoverage2100 9001 432.0
MaxCoverage2101 9001 426.0
MaxCoverage2101 9001 436.0
MaxCoverage2101 9001 437.0
MaxCoverage2101 9001 434.0
MaxCoverage2101 9001 437.0
MaxCoverage2101 9001 435.0
MaxCoverage2101 9001 437.0
MaxCoverage2101 9001 434.0
MaxCoverage2101 9001 439.0
MaxCoverage2101 9001 434.0
MaxCoverage2102 9001 549.0
MaxCoverage2102 9001 556.0
MaxCoverage2102 9001 553.0
MaxCoverage2102 9001 547.0
MaxCoverage2102 9001 552.0
MaxCoverage2102 9001 550.0
MaxCoverage2102 9001 553.0
MaxCoverage2102 9001 556.0
MaxCoverage2102 9001 554.0
MaxCoverage2102 9001 557.0
MaxCoverage2103 9001 673.0
MaxCoverage2103 9001 710.0
MaxCoverage2103 9001 697.0
MaxCoverage2103 9001 691.0
MaxCoverage2103 9001 697.0
MaxCoverage2103 9001 693.0
MaxCoverage2103 9001 694.0
M

In [30]:
logger_grow = ioh.logger.Analyzer(
    root=os.getcwd(),                  # Store data in the current working directory
    folder_name="grow100-5_maxcoverage",       # in a folder named: 'my-experiment'
    algorithm_name="grow100-5_ea",    # meta-data for the algorithm used to generate these results
    store_positions=True               # store x-variables in the logged files
)

for i in range(2100,2127+1):
    problem = ioh.get_problem(i, problem_class=ioh.ProblemClass.GRAPH)
    problem.attach_logger(logger_grow)
    dim_pb = problem.meta_data.n_variables
    grow = Test(nb_it//100, 100, 5)
    run_experiment(problem, grow, n_runs=10)

logger_grow.close()

MaxCoverage2100 9901 420.0
MaxCoverage2100 9901 436.0
MaxCoverage2100 9901 439.0
MaxCoverage2100 9901 437.0
MaxCoverage2100 9901 439.0
MaxCoverage2100 9901 434.0
MaxCoverage2100 9901 437.0
MaxCoverage2100 9901 435.0
MaxCoverage2100 9901 435.0
MaxCoverage2100 9901 436.0
MaxCoverage2101 9901 415.0
MaxCoverage2101 9901 441.0
MaxCoverage2101 9901 438.0
MaxCoverage2101 9901 436.0
MaxCoverage2101 9901 435.0
MaxCoverage2101 9901 440.0
MaxCoverage2101 9901 436.0
MaxCoverage2101 9901 438.0
MaxCoverage2101 9901 442.0
MaxCoverage2101 9901 440.0
MaxCoverage2102 9901 529.0
MaxCoverage2102 9901 558.0
MaxCoverage2102 9901 558.0
MaxCoverage2102 9901 555.0
MaxCoverage2102 9901 563.0
MaxCoverage2102 9901 559.0
MaxCoverage2102 9901 552.0
MaxCoverage2102 9901 555.0
MaxCoverage2102 9901 563.0
MaxCoverage2102 9901 557.0
MaxCoverage2103 9901 659.0
MaxCoverage2103 9901 697.0
MaxCoverage2103 9901 706.0
MaxCoverage2103 9901 716.0
MaxCoverage2103 9901 711.0
MaxCoverage2103 9901 698.0
MaxCoverage2103 9901 696.0
M

In [31]:
logger_grow = ioh.logger.Analyzer(
    root=os.getcwd(),                  # Store data in the current working directory
    folder_name="grow100-10_maxcoverage",       # in a folder named: 'my-experiment'
    algorithm_name="grow100-10_ea",    # meta-data for the algorithm used to generate these results
    store_positions=True               # store x-variables in the logged files
)

for i in range(2100,2127+1):
    problem = ioh.get_problem(i, problem_class=ioh.ProblemClass.GRAPH)
    problem.attach_logger(logger_grow)
    dim_pb = problem.meta_data.n_variables
    grow = Test(nb_it//100, 100, 10)
    run_experiment(problem, grow, n_runs=10)

logger_grow.close()

MaxCoverage2100 9901 421.0
MaxCoverage2100 9901 434.0
MaxCoverage2100 9901 438.0
MaxCoverage2100 9901 434.0
MaxCoverage2100 9901 437.0
MaxCoverage2100 9901 437.0
MaxCoverage2100 9901 432.0
MaxCoverage2100 9901 440.0
MaxCoverage2100 9901 439.0
MaxCoverage2100 9901 437.0
MaxCoverage2101 9901 420.0
MaxCoverage2101 9901 438.0
MaxCoverage2101 9901 442.0
MaxCoverage2101 9901 438.0
MaxCoverage2101 9901 437.0
MaxCoverage2101 9901 442.0
MaxCoverage2101 9901 445.0
MaxCoverage2101 9901 436.0
MaxCoverage2101 9901 440.0
MaxCoverage2101 9901 439.0
MaxCoverage2102 9901 541.0
MaxCoverage2102 9901 556.0
MaxCoverage2102 9901 561.0
MaxCoverage2102 9901 562.0
MaxCoverage2102 9901 555.0
MaxCoverage2102 9901 555.0
MaxCoverage2102 9901 552.0
MaxCoverage2102 9901 558.0
MaxCoverage2102 9901 556.0
MaxCoverage2102 9901 557.0
MaxCoverage2103 9901 645.0
MaxCoverage2103 9901 696.0
MaxCoverage2103 9901 706.0
MaxCoverage2103 9901 704.0
MaxCoverage2103 9901 703.0
MaxCoverage2103 9901 702.0
MaxCoverage2103 9901 705.0
M

In [32]:
logger_grow = ioh.logger.Analyzer(
    root=os.getcwd(),                  # Store data in the current working directory
    folder_name="grow10-5_maxcoverage",       # in a folder named: 'my-experiment'
    algorithm_name="grow10-5_ea",    # meta-data for the algorithm used to generate these results
    store_positions=True               # store x-variables in the logged files
)

for i in range(2100,2127+1):
    problem = ioh.get_problem(i, problem_class=ioh.ProblemClass.GRAPH)
    problem.attach_logger(logger_grow)
    dim_pb = problem.meta_data.n_variables
    grow = Test(nb_it//10, 10, 5)
    run_experiment(problem, grow, n_runs=10)

logger_grow.close()

MaxCoverage2100 9991 409.0
MaxCoverage2100 9991 421.0
MaxCoverage2100 9991 422.0
MaxCoverage2100 9991 416.0
MaxCoverage2100 9991 429.0
MaxCoverage2100 9991 419.0
MaxCoverage2100 9991 425.0
MaxCoverage2100 9991 425.0
MaxCoverage2100 9991 429.0
MaxCoverage2100 9991 426.0
MaxCoverage2101 9991 400.0
MaxCoverage2101 9991 421.0
MaxCoverage2101 9991 424.0
MaxCoverage2101 9991 422.0
MaxCoverage2101 9991 421.0
MaxCoverage2101 9991 421.0
MaxCoverage2101 9991 421.0
MaxCoverage2101 9991 422.0
MaxCoverage2101 9991 429.0
MaxCoverage2101 9991 430.0
MaxCoverage2102 9991 521.0
MaxCoverage2102 9991 539.0
MaxCoverage2102 9991 532.0
MaxCoverage2102 9991 529.0
MaxCoverage2102 9991 539.0
MaxCoverage2102 9991 543.0
MaxCoverage2102 9991 532.0
MaxCoverage2102 9991 544.0
MaxCoverage2102 9991 537.0
MaxCoverage2102 9991 533.0
MaxCoverage2103 9991 631.0
MaxCoverage2103 9991 669.0
MaxCoverage2103 9991 664.0
MaxCoverage2103 9991 664.0
MaxCoverage2103 9991 673.0
MaxCoverage2103 9991 673.0
MaxCoverage2103 9991 675.0
M

#### BigJumps

In [45]:
logger_grow = ioh.logger.Analyzer(
    root=os.getcwd(),                  # Store data in the current working directory
    folder_name="bigjumps10-5_maxcut",       # in a folder named: 'my-experiment'
    algorithm_name="bigjumps10-5_ea",    # meta-data for the algorithm used to generate these results
    store_positions=True               # store x-variables in the logged files
)

for i in range(2000,2004+1):
    problem = ioh.get_problem(i, problem_class=ioh.ProblemClass.GRAPH)
    problem.attach_logger(logger_grow)
    dim_pb = problem.meta_data.n_variables
    eabj = EABJ(nb_it//10, 10, 4, 1/dim)
    run_experiment(problem, eabj, n_runs=10)

logger_grow.close()

MaxCut2000 9991 11220.0
MaxCut2000 9991 11192.0
MaxCut2000 9991 11226.0
MaxCut2000 9991 11273.0
MaxCut2000 9991 11200.0
MaxCut2000 9991 11262.0
MaxCut2000 9991 11258.0
MaxCut2000 9991 11270.0
MaxCut2000 9991 11254.0
MaxCut2000 9991 11218.0
MaxCut2001 9991 11150.0
MaxCut2001 9991 11186.0
MaxCut2001 9991 11244.0
MaxCut2001 9991 11218.0
MaxCut2001 9991 11204.0
MaxCut2001 9991 11147.0
MaxCut2001 9991 11246.0
MaxCut2001 9991 11207.0
MaxCut2001 9991 11185.0
MaxCut2001 9991 11240.0
MaxCut2002 9991 11216.0
MaxCut2002 9991 11247.0
MaxCut2002 9991 11232.0
MaxCut2002 9991 11300.0
MaxCut2002 9991 11210.0
MaxCut2002 9991 11214.0
MaxCut2002 9991 11170.0
MaxCut2002 9991 11228.0
MaxCut2002 9991 11208.0
MaxCut2002 9991 11200.0
MaxCut2003 9991 11238.0
MaxCut2003 9991 11315.0
MaxCut2003 9991 11242.0
MaxCut2003 9991 11246.0
MaxCut2003 9991 11201.0
MaxCut2003 9991 11222.0
MaxCut2003 9991 11218.0
MaxCut2003 9991 11244.0
MaxCut2003 9991 11261.0
MaxCut2003 9991 11280.0
MaxCut2004 9991 11265.0
MaxCut2004 9991 

In [46]:
logger_grow = ioh.logger.Analyzer(
    root=os.getcwd(),                  # Store data in the current working directory
    folder_name="bigjumps10-5_maxcoverage",       # in a folder named: 'my-experiment'
    algorithm_name="bigjumps10-5_ea",    # meta-data for the algorithm used to generate these results
    store_positions=True               # store x-variables in the logged files
)

for i in range(2100,2127+1):
    problem = ioh.get_problem(i, problem_class=ioh.ProblemClass.GRAPH)
    problem.attach_logger(logger_grow)
    dim_pb = problem.meta_data.n_variables
    eabj = EABJ(nb_it//10, 10, 4, 1/dim)
    run_experiment(problem, eabj, n_runs=10)

logger_grow.close()

MaxCoverage2100 9991 412.0
MaxCoverage2100 9991 421.0
MaxCoverage2100 9991 401.0
MaxCoverage2100 9991 423.0
MaxCoverage2100 9991 415.0
MaxCoverage2100 9991 413.0
MaxCoverage2100 9991 422.0
MaxCoverage2100 9991 411.0
MaxCoverage2100 9991 413.0
MaxCoverage2100 9991 416.0
MaxCoverage2101 9991 415.0
MaxCoverage2101 9991 419.0
MaxCoverage2101 9991 418.0
MaxCoverage2101 9991 415.0
MaxCoverage2101 9991 412.0
MaxCoverage2101 9991 419.0
MaxCoverage2101 9991 423.0
MaxCoverage2101 9991 422.0
MaxCoverage2101 9991 421.0
MaxCoverage2101 9991 416.0
MaxCoverage2102 9991 542.0
MaxCoverage2102 9991 526.0
MaxCoverage2102 9991 514.0
MaxCoverage2102 9991 519.0
MaxCoverage2102 9991 520.0
MaxCoverage2102 9991 517.0
MaxCoverage2102 9991 520.0
MaxCoverage2102 9991 532.0
MaxCoverage2102 9991 518.0
MaxCoverage2102 9991 525.0
MaxCoverage2103 9991 -1.0
MaxCoverage2103 9991 -1.0
MaxCoverage2103 9991 -6.0
MaxCoverage2103 9991 -9.0
MaxCoverage2103 9991 -3.0
MaxCoverage2103 9991 563.0
MaxCoverage2103 9991 -7.0
MaxCove

# Bloc-note

Tester budget max et diminuer  
Descente de gradient stochastique // Relaxation  
Implémenter un reset pour la croissance